# Movie Recommendation System

## Introduction.


### About us








**ND2** is a medium size data science consultancy company based in Capetown. We provide data science solutions ranging from Data collection, data cleaning, analysis and model building and model deployment. We pride ourselves in quality delivery and offer services to financial I, government entities and government entities.


> ### Problem Statement
 

>A movie recommender system is a tool that suggests movies to users based on their viewing history and preferences. The goal of a movie recommender system is to provide personalized recommendations to users, by predicting the movies they are most likely to enjoy. This is typically done by analyzing large amounts of data on users and movies, such as ratings, reviews, and viewing history, and using machine learning algorithms to identify patterns and make predictions. Some popular techniques used in movie recommender systems include collaborative filtering, content-based filtering and hybrid approach which combines both the above techniques.

>In today’s technology driven world, recommender systems are socially and economically critical for ensuring that individuals can make appropriate choices surrounding the content they engage with on a daily basis. One application where this is especially true surrounds movie content recommendations; where intelligent algorithms can help viewers find great titles from tens of thousands of options.

>Recommender systems are needed because they can help users find relevant and interesting items among a large and growing collection of options. Without a recommender system, users may have a hard time discovering new items that match their interests, and may rely on simple, less effective methods such as browsing through lists or searching with keywords.

#### Possible Features of a Movie Recommender System:

>User profiling: This feature allows the system to create a profile for each user, which includes information such as their viewing history, ratings, and preferences.

>Content-based filtering: This feature recommends movies to users based on the attributes of the movies they have previously watched or liked.

>Collaborative filtering: This feature recommends movies to users based on the viewing history and preferences of similar users.

>Hybrid approach: This feature combines the above two techniques to provide more accurate recommendations.

>Real-time recommendations: This feature makes recommendations in real-time, so that the user doesn't have to wait for the system to process their request.

>Scalability: This feature allows the system to handle large amounts of data and make accurate recommendations for a large number of users.

>Continual learning: This feature allows the system to continuously learn and improve its recommendations over time.

>Handling new movies and users: This feature allows the system to handle new movies and users, and make recommendations for them as well.

>User-friendly Interface: This feature provides an easy to use interface to interact with the system, and make it accessible to a wide range of users.

### Data Source
For the purpose of this project; MovieLens dataset has been used. The data for the MovieLens dataset is maintained by the GroupLens research group in the Department of Computer Science and Engineering at the University of Minnesota. Additional movie content data was legally scraped from IMDB


### Table of Contents

1. Connecting to comet_ml

2. Importing Packages

3. Loading Data

4. Exploratory Data Analysis (EDA)

5. Feature Engineering

6. Model Training

7. Model Evaluation

8. Model Deployment
9. Conclusion

###   Create a connection to comet_ml

### Importing Relevant Libraries

In [ ]:
# Import our regular old heroes 
import numpy as np
import pandas as pd
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficientcy. 
import matplotlib.pyplot as plt
import seaborn as sns

# Entity featurization and similarity computation
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import TfidfVectorizer

# Libraries used during sorting procedures.
import operator # <-- Convienient item retrieval during iteration 
import heapq # <-- Efficient sorting of large lists

# Imported for our sanity
import warnings
warnings.filterwarnings('ignore')

### Loading Data

### Data Preprocessing

In [ ]:
# Cleaning and transforming the data in the input files to a format that can be used for building the model.

### Exploratory Data Analysis (EDA)

In [ ]:
# Understanding the characteristics of the data and identifying patterns or trends

### Feature Engineering

In [ ]:
# Extracting relevant features from the data that can be used to train the model

### Model Training 

In [ ]:
# Discussion on the model architecture
#Train a recommendation model using the engineered features, 

### Model evaluation

In [ ]:
# Evaluating the performance of the trained model using a suitable evaluation metric such as RMSE.

### Model deployment 

In [ ]:
# Deploying the trained model in a production environment to make recommendations to users using streamlit.

In [2]:
#End Comet experiment
#experiment.end()

### Conclusion

In [ ]:
# Summary of the key findings of the project and areas for future work.

### REFERENCE